In [11]:
import geopandas as gpd
import requests
from io import BytesIO
import zipfile
import os
import pandas as pd
from shapely.geometry import Point

In [55]:
projected_crs = 'EPSG:3857'

In [1186]:
url = 'https://echo.epa.gov/files/echodownloads/echo_exporter.zip'

In [1203]:
response = requests.get(url)
if response.status_code == 200:
    zip_file_bytes = BytesIO(response.content)

    with zipfile.ZipFile(zip_file_bytes, 'r') as zip_ref:
        print(zip_ref.namelist())
        csv_file_name = zip_ref.namelist()[0]
        zip_ref.extractall("echo_data")

    csv_path = os.path.join("echo_data", csv_file_name)
    facilities = pd.read_csv(csv_path)

    print(facilities.head())
else:
    print("Failed to download the file. Status code:", response.status_code)

['ECHO_EXPORTER.csv']


/var/folders/gr/tcb1m02d03zb8010bxpr4bfc0000gn/T/ipykernel_77152/996085387.py:11: DtypeWarning: Columns (2,3,5,11,12,18,20,29,41,54,55,56,57,62,64,67,69,70,71,72,73,74,79,81,84,86,89,90,91,96,98,101,103,104,105,108,110,114,115,117,119,122,123,124,127,130,131,132) have mixed types. Specify dtype option on import or set low_memory=False.
  facilities = pd.read_csv(csv_path)


    REGISTRY_ID                   FAC_NAME                    FAC_STREET  \
0  1.100524e+11           EDGAR L HOLT S/D                           NaN   
1  1.100540e+11  ALAN GARLAND FUNERAL HOME                           NaN   
2  1.100059e+11   HELLRUNG CONSTRUCTION CO                   ONE RIDGE B   
3  1.100708e+11       MIRANDY PRODUCTS LLC  825 BRICKELL BAY DR STE 1643   
4  1.100715e+11          ARDIE TAVANGARIAN              1816 CARLA RIDGE   

        FAC_CITY FAC_STATE FAC_ZIP   FAC_COUNTY  FAC_FIPS_CODE  \
0            NaN        NC     NaN          NaN            NaN   
1            NaN        IL     NaN          NaN            NaN   
2          ALTON        IL   62002      MADISON        17119.0   
3          MIAMI        FL   33131         DADE        12086.0   
4  BEVERLY HILLS        CA   90210  LOS ANGELES         6037.0   

   FAC_EPA_REGION FAC_INDIAN_CNTRY_FLG  ... FAC_DATE_LAST_INSPECTION_EPA  \
0             4.0                    N  ...                          N

In [1204]:
facilities = facilities[facilities['FAC_STATE'] == 'NC']

In [1400]:
facilities_filtered = facilities[
    (facilities['CAA_PERMIT_TYPES'].str.contains('Major', na=False)) |
    (facilities['CWA_PERMIT_TYPES'].str.contains('Major', na=False)) |
    (facilities['RCRA_PERMIT_TYPES'].notna())
]
facilities_filtered.to_csv("facilities_filtered_test.csv")

In [1209]:
facilities_filtered_geometry = [Point(xy) for xy in zip(facilities_filtered['FAC_LONG'], facilities_filtered['FAC_LAT'])]

In [1211]:
facilities_filtered_shapefile = gpd.GeoDataFrame(facilities_filtered, geometry=facilities_filtered_geometry)

In [1212]:
facilities_filtered_shapefile = facilities_filtered_shapefile.set_crs(epsg=4326, inplace=True)

In [1213]:
facilities_filtered_shapefile = facilities_filtered_shapefile.to_crs(projected_crs)

In [1214]:
facilities_buffered = facilities_filtered_shapefile.copy()
facilities_buffered['geometry'] = facilities_buffered.geometry.buffer(1609.34)

In [1215]:
# Bring in the Blocks file

In [1216]:
census = pd.read_csv("/Users/klausmayr/Desktop/ej/stateofexclusion/gis/DECENNIALPL2020/DECENNIALPL2020.P2-Data.csv")

/var/folders/gr/tcb1m02d03zb8010bxpr4bfc0000gn/T/ipykernel_77152/3466424946.py:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  census = pd.read_csv("/Users/klausmayr/Desktop/ej/stateofexclusion/gis/DECENNIALPL2020/DECENNIALPL2020.P2-Data.csv")


In [1217]:
blocks = gpd.read_file("/Users/klausmayr/Desktop/ej/stateofexclusion/gis/tl_2020_37_all/tl_2020_37_tabblock20.shp")

In [1218]:
census['GEOID20'] = census['GEO_ID'].str.slice(-15)

In [1219]:
census_blocks = pd.merge(blocks, census, on = "GEOID20", how = "left")

In [1220]:
columns_to_keep = [
    'COUNTYFP20',
    'GEOID20',
    'P2_001N',
    'P2_002N',
    'P2_003N',
    'P2_004N',
    'P2_005N',
    'P2_006N',
    'P2_007N',
    'P2_008N',
    'P2_009N',
    'P2_010N',
    'geometry']

In [1221]:
census_blocks = census_blocks[columns_to_keep]

In [1222]:
census_blocks = census_blocks.rename(columns={'geometry': 'geometry_blocks'})
census_blocks = census_blocks.set_geometry('geometry_blocks')

In [1223]:
census_blocks = census_blocks.to_crs(projected_crs)

In [1224]:
census_blocks['block_total_area'] = census_blocks.geometry.area

In [1225]:
blocks_buffers_joined = gpd.sjoin(census_blocks, facilities_buffered, how='inner', predicate='intersects')

In [1226]:
blocks_buffers_joined['geometry_buffer'] = facilities_buffered.loc[blocks_buffers_joined.index_right].geometry.values

In [1313]:
# This calculates the area of intersection between every census block and the EPA polluter 1 mile buffers
blocks_buffers_joined['area_of_block_buffer_intersection'] = blocks_buffers_joined.apply(lambda row: row['geometry_blocks'].intersection(row['geometry_buffer']).area, axis=1)
blocks_buffers_joined

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,FAC_DATE_LAST_FORMAL_ACT_ST,FAC_DATE_LAST_INFORMAL_ACT_EPA,FAC_DATE_LAST_INFORMAL_ACT_ST,FAC_FEDERAL_AGENCY,TRI_REPORTER,FAC_IMP_WATER_FLG,EJSCREEN_FLAG_US,geometry_buffer,area_of_cluster_muni_intersection,area_of_block_buffer_intersection
0,021,370210007001027,63,5,58,46,28,17,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,"POLYGON ((-9187933.737 4243133.416, -9187941.4...",7.865332e+03,7.865332e+03
41,021,370210009002007,505,10,495,468,283,176,6,3,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,"POLYGON ((-9187933.737 4243133.416, -9187941.4...",5.220093e+05,5.220093e+05
6047,021,370210009001019,49,6,43,38,25,13,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,"POLYGON ((-9187933.737 4243133.416, -9187941.4...",3.076737e+04,3.076737e+04
6614,021,370210009001001,49,1,48,48,48,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,"POLYGON ((-9187933.737 4243133.416, -9187941.4...",1.837904e+04,1.837904e+04
11447,021,370210009003010,529,43,486,472,160,285,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,Y,"POLYGON ((-9187933.737 4243133.416, -9187941.4...",3.262182e+05,3.262182e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233210,155,371559602022020,19,7,12,11,6,0,4,0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,"POLYGON ((-8793601.250 4135789.656, -8793608.9...",1.124918e+05,1.124918e+05
175653,077,370779706011018,92,6,86,86,35,50,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,"POLYGON ((-8753766.015 4322312.759, -8753773.7...",2.281643e+02,2.281643e+02
190209,077,370779706011023,81,4,77,77,55,22,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,"POLYGON ((-8753766.015 4322312.759, -8753773.7...",4.228312e+06,4.228312e+06
202809,077,370779706011021,149,0,149,149,123,26,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,N,"POLYGON ((-8753766.015 4322312.759, -8753773.7...",3.894663e+06,3.894663e+06


In [1314]:
intersection_areas = blocks_buffers_joined.groupby(blocks_buffers_joined.index)['area_of_block_buffer_intersection'].sum()

In [1315]:
census_blocks = census_blocks.merge(intersection_areas.rename('area_of_block_buffer_intersection'), left_index=True, right_index=True, how='left')

In [1316]:
census_blocks['block_buffer_overlap_percent'] = (census_blocks['area_of_block_buffer_intersection'] / census_blocks['block_total_area'])

In [1317]:
census_blocks['block_buffer_overlap_percent'].fillna(0, inplace=True)

In [1318]:
# Makes the maximum percent overlap 100%, because some were higher due to overlapping with multiple buffers
census_blocks['block_buffer_overlap_percent'] = census_blocks['block_buffer_overlap_percent'].clip(upper=1)

In [1319]:
census_blocks['block_buffer_overlap_percent'] = pd.to_numeric(census_blocks['block_buffer_overlap_percent'])

In [1320]:
census_blocks['P2_001N'] = pd.to_numeric(census_blocks['P2_001N'])
census_blocks['P2_002N'] = pd.to_numeric(census_blocks['P2_002N'])
census_blocks['P2_003N'] = pd.to_numeric(census_blocks['P2_003N'])
census_blocks['P2_004N'] = pd.to_numeric(census_blocks['P2_004N'])
census_blocks['P2_005N'] = pd.to_numeric(census_blocks['P2_005N'])
census_blocks['P2_006N'] = pd.to_numeric(census_blocks['P2_006N'])
census_blocks['P2_007N'] = pd.to_numeric(census_blocks['P2_007N'])
census_blocks['P2_008N'] = pd.to_numeric(census_blocks['P2_008N'])
census_blocks['P2_009N'] = pd.to_numeric(census_blocks['P2_009N'])
census_blocks['P2_010N'] = pd.to_numeric(census_blocks['P2_010N'])

In [1321]:
census_blocks['Total_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_001N']
census_blocks['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_002N']
census_blocks['Total_NH_White_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_005N']
census_blocks['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_006N']
census_blocks['Total_NH_Asian_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_008N']
census_blocks['Total_NH_Native_American_Pop_in_Prox_of_Polluter'] = census_blocks['block_buffer_overlap_percent'] * census_blocks['P2_007N']

In [1322]:
census_blocks.head().transpose()

,0,1,2,3,4
COUNTYFP20,021,021,021,021,021
GEOID20,370210007001027,370210026031032,370210026082009,370210005003000,370210028043012
P2_001N,63,13,82,16,8
P2_002N,5,6,5,3,2
P2_003N,58,7,77,13,6
P2_004N,46,7,72,13,3
P2_005N,28,7,71,13,3
P2_006N,17,0,0,0,0
P2_007N,0,0,0,0,0
P2_008N,0,0,1,0,0


In [1323]:
# Bring in SE Clusters layer

In [1324]:
census_blocks_for_cluster_join = census_blocks

In [1325]:
SE_Clusters = gpd.read_file('/Users/klausmayr/Desktop/ej/stateofexclusion/gis/SE_Clusters_reprojected_fixed_geometries.shp')

In [1326]:
SE_Clusters = SE_Clusters.rename(columns={'geometry': 'geometry_clusters', 'OBJECTID': 'cluster_id'})
SE_Clusters = SE_Clusters.set_geometry('geometry_clusters')
SE_Clusters

,cluster_id,geometry_clusters
0,25,"POLYGON ((-8734802.400 4015309.266, -8734957.4..."
1,66,"POLYGON ((-8748077.250 4017910.091, -8748078.8..."
2,71,"POLYGON ((-8729659.329 4017601.958, -8729825.0..."
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0..."
4,97,"POLYGON ((-8731843.417 4020804.422, -8731844.4..."
...,...,...
4262,12943,"POLYGON ((-8554108.937 4373513.298, -8554064.7..."
4263,12945,"POLYGON ((-8697493.228 4374416.563, -8697461.7..."
4264,12951,"POLYGON ((-8663873.517 4375542.842, -8663897.3..."
4265,12952,"POLYGON ((-8661725.496 4370374.212, -8661724.6..."


In [1327]:
census_blocks_for_cluster_join

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,block_total_area,area_of_cluster_muni_intersection,block_buffer_overlap_percent,Total_Pop_in_Prox_of_Polluter,Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter,Total_NH_White_Pop_in_Prox_of_Polluter,Total_NH_Black_AA_Pop_in_Prox_of_Polluter,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection
0,021,370210007001027,63,5,58,46,28,17,0,0,...,9.438215e+04,4.738615e+06,1.0,63.0,5.0,28.0,17.0,0.0,0.0,4.738615e+06
1,021,370210026031032,13,6,7,7,7,0,0,0,...,7.469069e+04,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,021,370210026082009,82,5,77,72,71,0,0,1,...,2.511728e+05,4.995293e+05,1.0,82.0,5.0,71.0,0.0,1.0,0.0,4.995293e+05
3,021,370210005003000,16,3,13,13,13,0,0,0,...,1.508198e+05,1.332759e+06,1.0,16.0,3.0,13.0,0.0,0.0,0.0,1.332759e+06
4,021,370210028043012,8,2,6,3,3,0,0,0,...,1.110626e+04,2.221253e+04,1.0,8.0,2.0,3.0,0.0,0.0,0.0,2.221253e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240227,071,370710329004013,17,2,15,14,14,0,0,0,...,4.225392e+04,3.034900e+05,1.0,17.0,2.0,14.0,0.0,0.0,0.0,3.034900e+05
240228,071,370710325091015,198,0,198,195,174,17,0,4,...,1.353829e+06,3.582669e+06,1.0,198.0,0.0,174.0,17.0,4.0,0.0,3.582669e+06
240229,041,370419301021041,36,2,34,31,23,2,0,0,...,1.676903e+05,4.588322e+05,1.0,36.0,2.0,23.0,2.0,0.0,0.0,4.588322e+05
240230,141,371419204032004,7,0,7,5,0,3,0,1,...,6.389169e+05,5.482029e+06,1.0,7.0,0.0,0.0,3.0,1.0,0.0,5.482029e+06


In [1328]:
# Determine municipal overlap

In [1329]:
# Get the area of the clusters
SE_Clusters['cluster_total_area'] = SE_Clusters.geometry.area

In [1330]:
# Read the municipal shapefile
muni = gpd.read_file('/Users/klausmayr/Desktop/ej/stateofexclusion/gis/NCDOT_City_Boundaries_reprojected_fixedgeometry.shp')

muni_columns_to_keep = [
       'MunicipalB',
        'geometry']

muni = muni[muni_columns_to_keep]

muni = muni.rename(columns={'geometry': 'geometry_muni'})
muni = muni.set_geometry('geometry_muni')

muni

,MunicipalB,geometry_muni
0,Mars Hill,"MULTIPOLYGON (((-9190343.116 4276351.245, -919..."
1,Biltmore Forest,"MULTIPOLYGON (((-9188694.015 4240895.214, -918..."
2,Black Mountain,"POLYGON ((-9163416.364 4251081.358, -9163245.0..."
3,Drexel,"MULTIPOLYGON (((-9083583.018 4268180.694, -908..."
4,Connelly Springs,"POLYGON ((-9068651.480 4270691.794, -9068660.8..."
...,...,...
545,Hendersonville,"MULTIPOLYGON (((-9174445.448 4212839.739, -917..."
546,Maggie Valley,"MULTIPOLYGON (((-9249356.696 4233791.811, -924..."
547,Sanford,"MULTIPOLYGON (((-8813598.726 4236603.191, -881..."
548,Garner,"MULTIPOLYGON (((-8756428.255 4263714.124, -875..."


In [1331]:
SE_Clusters_muni_overlap = gpd.sjoin(SE_Clusters, muni, how='inner', predicate='intersects')
SE_Clusters_muni_overlap = SE_Clusters_muni_overlap.rename(columns={'index_right': 'municipality_index'})
SE_Clusters_muni_overlap

,cluster_id,geometry_clusters,cluster_total_area,municipality_index,MunicipalB
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,517,Southport
8,145,"POLYGON ((-8728696.972 4026000.710, -8728656.4...",4.743227e+06,516,Shallotte
9,154,"POLYGON ((-8725818.695 4027215.939, -8725782.8...",7.327717e+05,516,Shallotte
12,168,"POLYGON ((-8727323.178 4029142.957, -8727342.8...",8.576853e+06,516,Shallotte
13,169,"POLYGON ((-8724750.250 4029907.436, -8724739.5...",3.532106e+06,516,Shallotte
...,...,...,...,...,...
4259,12930,"POLYGON ((-8610150.397 4374128.951, -8610192.9...",2.862866e+07,406,Seaboard
4241,12844,"POLYGON ((-8972458.827 4370785.677, -8972475.4...",2.471892e+05,77,Mount Airy
4245,12849,"POLYGON ((-8975135.281 4370499.824, -8975134.8...",5.181774e+05,77,Mount Airy
4250,12874,"POLYGON ((-8574998.596 4367242.552, -8575062.3...",4.638998e+07,409,Como


In [1332]:
SE_Clusters_muni_overlap = SE_Clusters_muni_overlap.set_geometry('geometry_clusters')

In [1333]:
SE_Clusters_muni_overlap['geometry_muni'] = muni.loc[SE_Clusters_muni_overlap.municipality_index].geometry.values
SE_Clusters_muni_overlap

,cluster_id,geometry_clusters,cluster_total_area,municipality_index,MunicipalB,geometry_muni
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,517,Southport,"MULTIPOLYGON (((-8684081.687 4020881.116, -868..."
8,145,"POLYGON ((-8728696.972 4026000.710, -8728656.4...",4.743227e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872..."
9,154,"POLYGON ((-8725818.695 4027215.939, -8725782.8...",7.327717e+05,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872..."
12,168,"POLYGON ((-8727323.178 4029142.957, -8727342.8...",8.576853e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872..."
13,169,"POLYGON ((-8724750.250 4029907.436, -8724739.5...",3.532106e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872..."
...,...,...,...,...,...,...
4259,12930,"POLYGON ((-8610150.397 4374128.951, -8610192.9...",2.862866e+07,406,Seaboard,"POLYGON ((-8621835.480 4367323.421, -8621790.2..."
4241,12844,"POLYGON ((-8972458.827 4370785.677, -8972475.4...",2.471892e+05,77,Mount Airy,"MULTIPOLYGON (((-8976110.757 4363554.998, -897..."
4245,12849,"POLYGON ((-8975135.281 4370499.824, -8975134.8...",5.181774e+05,77,Mount Airy,"MULTIPOLYGON (((-8976110.757 4363554.998, -897..."
4250,12874,"POLYGON ((-8574998.596 4367242.552, -8575062.3...",4.638998e+07,409,Como,"POLYGON ((-8570707.033 4368697.465, -8570790.9..."


In [1334]:
SE_Clusters_muni_overlap['area_of_cluster_muni_intersection'] = SE_Clusters_muni_overlap.apply(lambda row: row['geometry_clusters'].intersection(row['geometry_muni']).area, axis=1)
SE_Clusters_muni_overlap

,cluster_id,geometry_clusters,cluster_total_area,municipality_index,MunicipalB,geometry_muni,area_of_cluster_muni_intersection
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,517,Southport,"MULTIPOLYGON (((-8684081.687 4020881.116, -868...",9.393677e+04
8,145,"POLYGON ((-8728696.972 4026000.710, -8728656.4...",4.743227e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872...",5.300812e+05
9,154,"POLYGON ((-8725818.695 4027215.939, -8725782.8...",7.327717e+05,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872...",5.813009e+05
12,168,"POLYGON ((-8727323.178 4029142.957, -8727342.8...",8.576853e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872...",1.943536e+04
13,169,"POLYGON ((-8724750.250 4029907.436, -8724739.5...",3.532106e+06,516,Shallotte,"MULTIPOLYGON (((-8724677.888 4017354.095, -872...",2.115237e+06
...,...,...,...,...,...,...,...
4259,12930,"POLYGON ((-8610150.397 4374128.951, -8610192.9...",2.862866e+07,406,Seaboard,"POLYGON ((-8621835.480 4367323.421, -8621790.2...",3.608238e+04
4241,12844,"POLYGON ((-8972458.827 4370785.677, -8972475.4...",2.471892e+05,77,Mount Airy,"MULTIPOLYGON (((-8976110.757 4363554.998, -897...",2.471892e+05
4245,12849,"POLYGON ((-8975135.281 4370499.824, -8975134.8...",5.181774e+05,77,Mount Airy,"MULTIPOLYGON (((-8976110.757 4363554.998, -897...",5.147147e+05
4250,12874,"POLYGON ((-8574998.596 4367242.552, -8575062.3...",4.638998e+07,409,Como,"POLYGON ((-8570707.033 4368697.465, -8570790.9...",7.631569e+03


In [1335]:
muni_intersection_areas = SE_Clusters_muni_overlap.groupby(SE_Clusters_muni_overlap.cluster_id)['area_of_cluster_muni_intersection'].sum()
muni_intersection_areas

cluster_id
82       9.393677e+04
145      5.300812e+05
154      5.813009e+05
168      1.943536e+04
169      2.115237e+06
             ...     
12874    3.115362e+04
12877    7.070617e+05
12904    2.849918e+04
12930    3.608238e+04
12955    1.308790e+06
Name: area_of_cluster_muni_intersection, Length: 3055, dtype: float64

In [1336]:
SE_Clusters = SE_Clusters.merge(muni_intersection_areas.rename('area_of_cluster_muni_intersection'), left_index=True, right_index=True, how='left')
SE_Clusters

,cluster_id,geometry_clusters,cluster_total_area,area_of_cluster_muni_intersection
0,25,"POLYGON ((-8734802.400 4015309.266, -8734957.4...",9.590385e+04,NaN
1,66,"POLYGON ((-8748077.250 4017910.091, -8748078.8...",4.077530e+05,NaN
2,71,"POLYGON ((-8729659.329 4017601.958, -8729825.0...",9.030900e+05,NaN
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,NaN
4,97,"POLYGON ((-8731843.417 4020804.422, -8731844.4...",6.259919e+04,NaN
...,...,...,...,...
4262,12943,"POLYGON ((-8554108.937 4373513.298, -8554064.7...",1.470660e+07,NaN
4263,12945,"POLYGON ((-8697493.228 4374416.563, -8697461.7...",3.147954e+07,NaN
4264,12951,"POLYGON ((-8663873.517 4375542.842, -8663897.3...",6.045330e+06,NaN
4265,12952,"POLYGON ((-8661725.496 4370374.212, -8661724.6...",1.226132e+07,8785.830838


In [1337]:
SE_Clusters['cluster_muni_percent_overlap'] = (SE_Clusters['area_of_cluster_muni_intersection'] / SE_Clusters['cluster_total_area'])
SE_Clusters

,cluster_id,geometry_clusters,cluster_total_area,area_of_cluster_muni_intersection,cluster_muni_percent_overlap
0,25,"POLYGON ((-8734802.400 4015309.266, -8734957.4...",9.590385e+04,NaN,NaN
1,66,"POLYGON ((-8748077.250 4017910.091, -8748078.8...",4.077530e+05,NaN,NaN
2,71,"POLYGON ((-8729659.329 4017601.958, -8729825.0...",9.030900e+05,NaN,NaN
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,NaN,NaN
4,97,"POLYGON ((-8731843.417 4020804.422, -8731844.4...",6.259919e+04,NaN,NaN
...,...,...,...,...,...
4262,12943,"POLYGON ((-8554108.937 4373513.298, -8554064.7...",1.470660e+07,NaN,NaN
4263,12945,"POLYGON ((-8697493.228 4374416.563, -8697461.7...",3.147954e+07,NaN,NaN
4264,12951,"POLYGON ((-8663873.517 4375542.842, -8663897.3...",6.045330e+06,NaN,NaN
4265,12952,"POLYGON ((-8661725.496 4370374.212, -8661724.6...",1.226132e+07,8785.830838,0.000717


In [1338]:
SE_Clusters['cluster_muni_percent_overlap'].fillna(0, inplace=True)

In [1339]:
SE_Clusters['cluster_muni_percent_overlap'] = SE_Clusters['cluster_muni_percent_overlap'].clip(upper=1)

In [1340]:
SE_Clusters['cluster_muni_percent_overlap'] = pd.to_numeric(SE_Clusters['cluster_muni_percent_overlap'])

In [1341]:
# This labels Y if the overlap of the clusters with the municipal boundaries is less than 75%
SE_Clusters['cluster_with_less_than_50pct_overlap_with_municipality'] = SE_Clusters.apply(
    lambda row: 'Y' if 0 < row['cluster_muni_percent_overlap'] < 0.5 else None, axis=1
)
SE_Clusters

,cluster_id,geometry_clusters,cluster_total_area,area_of_cluster_muni_intersection,cluster_muni_percent_overlap,cluster_with_less_than_50pct_overlap_with_municipality
0,25,"POLYGON ((-8734802.400 4015309.266, -8734957.4...",9.590385e+04,NaN,0.000000,None
1,66,"POLYGON ((-8748077.250 4017910.091, -8748078.8...",4.077530e+05,NaN,0.000000,None
2,71,"POLYGON ((-8729659.329 4017601.958, -8729825.0...",9.030900e+05,NaN,0.000000,None
3,82,"POLYGON ((-8685573.582 4019368.636, -8685534.0...",9.393677e+04,NaN,0.000000,None
4,97,"POLYGON ((-8731843.417 4020804.422, -8731844.4...",6.259919e+04,NaN,0.000000,None
...,...,...,...,...,...,...
4262,12943,"POLYGON ((-8554108.937 4373513.298, -8554064.7...",1.470660e+07,NaN,0.000000,None
4263,12945,"POLYGON ((-8697493.228 4374416.563, -8697461.7...",3.147954e+07,NaN,0.000000,None
4264,12951,"POLYGON ((-8663873.517 4375542.842, -8663897.3...",6.045330e+06,NaN,0.000000,None
4265,12952,"POLYGON ((-8661725.496 4370374.212, -8661724.6...",1.226132e+07,8785.830838,0.000717,Y


In [1342]:
## At this point, need to join the cluster + municipality dataframe back with the census block dataframe
## But I only want the ones that fully overlap, don't want the ones that just intersect. Without this next step
## I get all of the blocks that border the cluster blocks.

# Spatial Join the Census Block shapefile (which includes demographic info) with the Clusters shapefile 
# (which includes info about how percent Clusters overlap with municipal boundaries 
census_block_clusters_joined = gpd.sjoin(census_blocks, SE_Clusters, how='inner', predicate='intersects')
census_block_clusters_joined = census_block_clusters_joined.rename(columns={'index_right': 'cluster_index'})
census_block_clusters_joined.head().transpose()

,7,17211,82366,84526,85863
COUNTYFP20,191,191,191,191,191
GEOID20,371910001043015,371910001021004,371910001024000,371910001024001,371910003042000
P2_001N,0,77,0,0,46
P2_002N,0,10,0,0,8
P2_003N,0,67,0,0,38
P2_004N,0,61,0,0,37
P2_005N,0,30,0,0,13
P2_006N,0,31,0,0,24
P2_007N,0,0,0,0,0
P2_008N,0,0,0,0,0


In [1343]:
# Add a column for the geometry of the clusters
census_block_clusters_joined['geometry_cluster'] = SE_Clusters.loc[census_block_clusters_joined.cluster_index].geometry.values
census_block_clusters_joined

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_index,cluster_id,cluster_total_area,area_of_cluster_muni_intersection_right,cluster_muni_percent_overlap,cluster_with_less_than_50pct_overlap_with_municipality,geometry_cluster
7,191,371910001043015,0,0,0,0,0,0,0,0,...,0.0,0.0,2.177813e+04,2048,6096,2.276703e+06,317200.542121,0.139325,Y,"POLYGON ((-8681129.708 4231032.126, -8681129.1..."
17211,191,371910001021004,77,10,67,61,30,31,0,0,...,0.0,0.0,6.188065e+05,2048,6096,2.276703e+06,317200.542121,0.139325,Y,"POLYGON ((-8681129.708 4231032.126, -8681129.1..."
82366,191,371910001024000,0,0,0,0,0,0,0,0,...,0.0,0.0,1.075963e+04,2048,6096,2.276703e+06,317200.542121,0.139325,Y,"POLYGON ((-8681129.708 4231032.126, -8681129.1..."
84526,191,371910001024001,0,0,0,0,0,0,0,0,...,0.0,0.0,1.505756e+05,2048,6096,2.276703e+06,317200.542121,0.139325,Y,"POLYGON ((-8681129.708 4231032.126, -8681129.1..."
85863,191,371910003042000,46,8,38,37,13,24,0,0,...,0.0,0.0,1.222240e+06,2048,6096,2.276703e+06,317200.542121,0.139325,Y,"POLYGON ((-8681129.708 4231032.126, -8681129.1..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224333,001,370010205021005,51,19,32,31,8,20,0,0,...,0.0,0.0,NaN,3819,11316,6.030756e+04,36458.746951,0.604547,None,"POLYGON ((-8844125.265 4316913.156, -8844139.9..."
214748,119,371190056231011,386,31,355,346,116,212,0,18,...,18.0,0.0,8.245144e+06,1462,4224,9.246773e+04,NaN,0.000000,None,"POLYGON ((-8988047.785 4202837.465, -8988047.8..."
235170,119,371190056231013,58,12,46,36,12,19,0,4,...,4.0,0.0,1.299306e+06,1462,4224,9.246773e+04,NaN,0.000000,None,"POLYGON ((-8988047.785 4202837.465, -8988047.8..."
218197,119,371190060052013,31,0,31,27,6,16,0,0,...,0.0,0.0,5.925754e+04,1460,4217,1.975250e+04,111721.520746,1.000000,None,"POLYGON ((-9014543.939 4202992.660, -9014577.1..."


In [1344]:
# Add column of the area of intersection between the blocks and the clusters. Remember, just looking for the ones
# that are pretty much fully intersecting
census_block_clusters_joined['block_cluster_intersection_area'] = census_block_clusters_joined.apply(lambda row: row['geometry_blocks'].intersection(row['geometry_cluster']).area, axis=1)

In [1345]:
census_block_clusters_joined['block_cluster_percent_overlap'] = (census_block_clusters_joined['block_cluster_intersection_area'] / census_block_clusters_joined['block_total_area'])

In [1346]:
census_block_clusters_joined = census_block_clusters_joined[census_block_clusters_joined['block_cluster_percent_overlap'] > 0.9]
census_block_clusters_joined.head().transpose()

,141037,8,71000,126489,137778
COUNTYFP20,191,191,191,191,191
GEOID20,371910001021006,371910006034011,371910006034005,371910006034004,371910006034006
P2_001N,48,34,23,44,34
P2_002N,4,9,10,2,10
P2_003N,44,25,13,42,24
P2_004N,44,25,12,38,23
P2_005N,7,3,3,5,8
P2_006N,34,22,9,30,15
P2_007N,0,0,0,0,0
P2_008N,3,0,0,0,0


In [1347]:
cluster_columns_to_keep = [
    'GEOID20',
       'cluster_id',
    'cluster_with_less_than_50pct_overlap_with_municipality'
]

census_block_clusters_to_join = census_block_clusters_joined[cluster_columns_to_keep]

In [1348]:
all_census_blocks_with_cluster_data = census_blocks.merge(census_block_clusters_to_join, on='GEOID20', how='outer')
all_census_blocks_with_cluster_data.head().transpose()

,0,1,2,3,4
COUNTYFP20,021,021,021,021,021
GEOID20,370210007001027,370210026031032,370210026082009,370210005003000,370210028043012
P2_001N,63,13,82,16,8
P2_002N,5,6,5,3,2
P2_003N,58,7,77,13,6
P2_004N,46,7,72,13,3
P2_005N,28,7,71,13,3
P2_006N,17,0,0,0,0
P2_007N,0,0,0,0,0
P2_008N,0,0,1,0,0


In [1349]:
# Now going to label which census blocks are within 1/10 of a mile of a municipality. Have to do this separately
# from the previous underbounded analysis step, because in the report their are data points that reference
# the proximity of all census blocks to municipalities

In [1350]:
all_census_blocks_with_cluster_data_for_underbounded_analysis = all_census_blocks_with_cluster_data

In [1351]:
muni

,MunicipalB,geometry_muni
0,Mars Hill,"MULTIPOLYGON (((-9190343.116 4276351.245, -919..."
1,Biltmore Forest,"MULTIPOLYGON (((-9188694.015 4240895.214, -918..."
2,Black Mountain,"POLYGON ((-9163416.364 4251081.358, -9163245.0..."
3,Drexel,"MULTIPOLYGON (((-9083583.018 4268180.694, -908..."
4,Connelly Springs,"POLYGON ((-9068651.480 4270691.794, -9068660.8..."
...,...,...
545,Hendersonville,"MULTIPOLYGON (((-9174445.448 4212839.739, -917..."
546,Maggie Valley,"MULTIPOLYGON (((-9249356.696 4233791.811, -924..."
547,Sanford,"MULTIPOLYGON (((-8813598.726 4236603.191, -881..."
548,Garner,"MULTIPOLYGON (((-8756428.255 4263714.124, -875..."


In [1352]:
muni_buffered = muni.copy()
muni_buffered['geometry_muni'] = muni_buffered.geometry.buffer(160.934)
muni_buffered

,MunicipalB,geometry_muni
0,Mars Hill,"MULTIPOLYGON (((-9186134.917 4278464.096, -918..."
1,Biltmore Forest,"POLYGON ((-9190347.978 4240857.698, -9190347.2..."
2,Black Mountain,"POLYGON ((-9169382.350 4246469.763, -9169380.9..."
3,Drexel,"MULTIPOLYGON (((-9086621.003 4267060.192, -908..."
4,Connelly Springs,"POLYGON ((-9074974.052 4264538.871, -9074980.2..."
...,...,...
545,Hendersonville,"MULTIPOLYGON (((-9172619.895 4203906.338, -917..."
546,Maggie Valley,"MULTIPOLYGON (((-9251965.422 4235116.249, -925..."
547,Sanford,"MULTIPOLYGON (((-8805230.906 4227120.546, -880..."
548,Garner,"MULTIPOLYGON (((-8741736.042 4258168.513, -874..."


In [1353]:
all_census_blocks_with_cluster_data_for_underbounded_analysis['near_muni'] = 'N'

for idx, block in all_census_blocks_with_cluster_data_for_underbounded_analysis.iterrows():
    if muni_buffered.intersects(block.geometry_blocks).any():
        all_census_blocks_with_cluster_data_for_underbounded_analysis.at[idx, 'near_muni'] = 'Y'

all_census_blocks_with_cluster_data_for_underbounded_analysis.head().transpose()

,0,1,2,3,4
COUNTYFP20,021,021,021,021,021
GEOID20,370210007001027,370210026031032,370210026082009,370210005003000,370210028043012
P2_001N,63,13,82,16,8
P2_002N,5,6,5,3,2
P2_003N,58,7,77,13,6
P2_004N,46,7,72,13,3
P2_005N,28,7,71,13,3
P2_006N,17,0,0,0,0
P2_007N,0,0,0,0,0
P2_008N,0,0,1,0,0


In [1354]:
white_muni = gpd.read_file('/Users/klausmayr/Desktop/ej/stateofexclusion/gis/majority_white_municipalities.shp')

In [1355]:
white_muni_buffered = white_muni.copy()
white_muni_buffered['geometry'] = white_muni_buffered.geometry.buffer(160.934)

In [1356]:
all_census_blocks_with_cluster_data_for_underbounded_analysis['near_white_muni'] = 'N'

for idx, block in all_census_blocks_with_cluster_data_for_underbounded_analysis.iterrows():
    if white_muni_buffered.intersects(block.geometry_blocks).any():
        all_census_blocks_with_cluster_data_for_underbounded_analysis.at[idx, 'near_white_muni'] = 'Y'

In [1357]:
all_census_blocks_with_cluster_data_for_underbounded_analysis

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter,Total_NH_White_Pop_in_Prox_of_Polluter,Total_NH_Black_AA_Pop_in_Prox_of_Polluter,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni
0,021,370210007001027,63,5,58,46,28,17,0,0,...,5.0,28.0,17.0,0.0,0.0,4.738615e+06,NaN,NaN,Y,Y
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,N,N
2,021,370210026082009,82,5,77,72,71,0,0,1,...,5.0,71.0,0.0,1.0,0.0,4.995293e+05,NaN,NaN,N,N
3,021,370210005003000,16,3,13,13,13,0,0,0,...,3.0,13.0,0.0,0.0,0.0,1.332759e+06,NaN,NaN,Y,Y
4,021,370210028043012,8,2,6,3,3,0,0,0,...,2.0,3.0,0.0,0.0,0.0,2.221253e+04,NaN,NaN,N,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241099,071,370710329004013,17,2,15,14,14,0,0,0,...,2.0,14.0,0.0,0.0,0.0,3.034900e+05,NaN,NaN,Y,Y
241100,071,370710325091015,198,0,198,195,174,17,0,4,...,0.0,174.0,17.0,4.0,0.0,3.582669e+06,NaN,NaN,Y,Y
241101,041,370419301021041,36,2,34,31,23,2,0,0,...,2.0,23.0,2.0,0.0,0.0,4.588322e+05,NaN,NaN,N,N
241102,141,371419204032004,7,0,7,5,0,3,0,1,...,0.0,0.0,3.0,1.0,0.0,5.482029e+06,677.0,None,Y,Y


In [1358]:
# Bring in Tier map

In [1359]:
tiers = gpd.read_file('/Users/klausmayr/Desktop/ej/stateofexclusion/gis/county_tiers.shp')
tiers

,tier,geometry
0,3,"MULTIPOLYGON (((-8678415.727 4008301.887, -867..."
1,1,"MULTIPOLYGON (((-8776901.985 4043379.574, -877..."
2,2,"MULTIPOLYGON (((-8568076.269 4137950.846, -856..."


In [1360]:
census_block_clusters_tiers = gpd.sjoin(all_census_blocks_with_cluster_data_for_underbounded_analysis, tiers, how='inner', predicate='intersects')
census_block_clusters_tiers

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_Black_AA_Pop_in_Prox_of_Polluter,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni,index_right,tier
0,021,370210007001027,63,5,58,46,28,17,0,0,...,17.0,0.0,0.0,4.738615e+06,NaN,NaN,Y,Y,0,3
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,0.0,0.0,NaN,NaN,NaN,N,N,0,3
2,021,370210026082009,82,5,77,72,71,0,0,1,...,0.0,1.0,0.0,4.995293e+05,NaN,NaN,N,N,0,3
3,021,370210005003000,16,3,13,13,13,0,0,0,...,0.0,0.0,0.0,1.332759e+06,NaN,NaN,Y,Y,0,3
4,021,370210028043012,8,2,6,3,3,0,0,0,...,0.0,0.0,0.0,2.221253e+04,NaN,NaN,N,N,0,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241091,071,370710307003012,131,3,128,120,112,6,0,0,...,6.0,0.0,0.0,2.818822e+06,NaN,NaN,Y,Y,2,2
241092,071,370710329004001,86,24,62,58,50,7,0,1,...,7.0,1.0,0.0,1.250891e+06,NaN,NaN,Y,Y,2,2
241098,071,370710313032032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,1.376583e+04,NaN,NaN,Y,Y,2,2
241099,071,370710329004013,17,2,15,14,14,0,0,0,...,0.0,0.0,0.0,3.034900e+05,NaN,NaN,Y,Y,2,2


In [1361]:
census_block_clusters_tiers = census_block_clusters_tiers.drop('index_right', axis=1)

In [1362]:
census_block_clusters_tiers

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_White_Pop_in_Prox_of_Polluter,Total_NH_Black_AA_Pop_in_Prox_of_Polluter,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni,tier
0,021,370210007001027,63,5,58,46,28,17,0,0,...,28.0,17.0,0.0,0.0,4.738615e+06,NaN,NaN,Y,Y,3
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,0.0,0.0,0.0,NaN,NaN,NaN,N,N,3
2,021,370210026082009,82,5,77,72,71,0,0,1,...,71.0,0.0,1.0,0.0,4.995293e+05,NaN,NaN,N,N,3
3,021,370210005003000,16,3,13,13,13,0,0,0,...,13.0,0.0,0.0,0.0,1.332759e+06,NaN,NaN,Y,Y,3
4,021,370210028043012,8,2,6,3,3,0,0,0,...,3.0,0.0,0.0,0.0,2.221253e+04,NaN,NaN,N,N,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
241091,071,370710307003012,131,3,128,120,112,6,0,0,...,112.0,6.0,0.0,0.0,2.818822e+06,NaN,NaN,Y,Y,2
241092,071,370710329004001,86,24,62,58,50,7,0,1,...,50.0,7.0,1.0,0.0,1.250891e+06,NaN,NaN,Y,Y,2
241098,071,370710313032032,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.376583e+04,NaN,NaN,Y,Y,2
241099,071,370710329004013,17,2,15,14,14,0,0,0,...,14.0,0.0,0.0,0.0,3.034900e+05,NaN,NaN,Y,Y,2


In [1363]:
regions = gpd.read_file('/Users/klausmayr/Desktop/ej/stateofexclusion/gis/county_regions.shp')

In [1364]:
census_block_clusters_tiers_regions = gpd.sjoin(census_block_clusters_tiers, regions, how='inner', predicate='intersects')

In [1365]:
census_block_clusters_tiers_regions

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni,tier,index_right,regions
0,021,370210007001027,63,5,58,46,28,17,0,0,...,0.0,0.0,4.738615e+06,NaN,NaN,Y,Y,3,0,Mountains
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,0.0,NaN,NaN,NaN,N,N,3,0,Mountains
2,021,370210026082009,82,5,77,72,71,0,0,1,...,1.0,0.0,4.995293e+05,NaN,NaN,N,N,3,0,Mountains
3,021,370210005003000,16,3,13,13,13,0,0,0,...,0.0,0.0,1.332759e+06,NaN,NaN,Y,Y,3,0,Mountains
4,021,370210028043012,8,2,6,3,3,0,0,0,...,0.0,0.0,2.221253e+04,NaN,NaN,N,N,3,0,Mountains
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240424,049,370499605022052,0,0,0,0,0,0,0,0,...,0.0,0.0,3.688166e+06,NaN,NaN,Y,Y,2,1,Coastal Plains
240425,049,370499601011037,0,0,0,0,0,0,0,0,...,0.0,0.0,8.752532e+04,NaN,NaN,Y,Y,2,1,Coastal Plains
240426,049,370499613021018,0,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,N,N,2,1,Coastal Plains
240427,049,370499601021010,0,0,0,0,0,0,0,0,...,0.0,0.0,NaN,NaN,NaN,N,N,2,1,Coastal Plains


In [1366]:
census_block_clusters_tiers_regions = census_block_clusters_tiers_regions.drop('index_right', axis=1)

In [1367]:
census_block_clusters_tiers_regions

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_Black_AA_Pop_in_Prox_of_Polluter,Total_NH_Asian_Pop_in_Prox_of_Polluter,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni,tier,regions
0,021,370210007001027,63,5,58,46,28,17,0,0,...,17.0,0.0,0.0,4.738615e+06,NaN,NaN,Y,Y,3,Mountains
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,0.0,0.0,NaN,NaN,NaN,N,N,3,Mountains
2,021,370210026082009,82,5,77,72,71,0,0,1,...,0.0,1.0,0.0,4.995293e+05,NaN,NaN,N,N,3,Mountains
3,021,370210005003000,16,3,13,13,13,0,0,0,...,0.0,0.0,0.0,1.332759e+06,NaN,NaN,Y,Y,3,Mountains
4,021,370210028043012,8,2,6,3,3,0,0,0,...,0.0,0.0,0.0,2.221253e+04,NaN,NaN,N,N,3,Mountains
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240424,049,370499605022052,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,3.688166e+06,NaN,NaN,Y,Y,2,Coastal Plains
240425,049,370499601011037,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,8.752532e+04,NaN,NaN,Y,Y,2,Coastal Plains
240426,049,370499613021018,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,NaN,NaN,N,N,2,Coastal Plains
240427,049,370499601021010,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,NaN,NaN,N,N,2,Coastal Plains


In [1368]:
# Finding the predominant region and tier

In [1369]:
def predominant_value(df, column):
    return df.groupby(column)['block_total_area'].sum().idxmax()

In [1370]:
census_block_clusters_tiers_regions['Cluster_Tier'] = None
census_block_clusters_tiers_regions['Cluster_Region'] = None

In [1371]:
for obj_id in census_block_clusters_tiers_regions['cluster_id'].unique():
    cluster_blocks = census_block_clusters_tiers_regions[census_block_clusters_tiers_regions['cluster_id'] == obj_id]
    
    if not cluster_blocks.empty:
        # Determine predominant tier
        predominant_tier = predominant_value(cluster_blocks, 'tier')
        census_block_clusters_tiers_regions.loc[census_block_clusters_tiers_regions['cluster_id'] == obj_id, 'Cluster_Tier'] = predominant_tier
        
        # Determine predominant region
        predominant_region = predominant_value(cluster_blocks, 'regions')
        census_block_clusters_tiers_regions.loc[census_block_clusters_tiers_regions['cluster_id'] == obj_id, 'Cluster_Region'] = predominant_region

In [1372]:
census_block_clusters_tiers_regions.head(50)

,COUNTYFP20,GEOID20,P2_001N,P2_002N,P2_003N,P2_004N,P2_005N,P2_006N,P2_007N,P2_008N,...,Total_NH_Native_American_Pop_in_Prox_of_Polluter,area_of_block_buffer_intersection,cluster_id,cluster_with_less_than_50pct_overlap_with_municipality,near_muni,near_white_muni,tier,regions,Cluster_Tier,Cluster_Region
0,021,370210007001027,63,5,58,46,28,17,0,0,...,0.0,4.738615e+06,NaN,NaN,Y,Y,3,Mountains,None,None
1,021,370210026031032,13,6,7,7,7,0,0,0,...,0.0,NaN,NaN,NaN,N,N,3,Mountains,None,None
2,021,370210026082009,82,5,77,72,71,0,0,1,...,0.0,4.995293e+05,NaN,NaN,N,N,3,Mountains,None,None
3,021,370210005003000,16,3,13,13,13,0,0,0,...,0.0,1.332759e+06,NaN,NaN,Y,Y,3,Mountains,None,None
4,021,370210028043012,8,2,6,3,3,0,0,0,...,0.0,2.221253e+04,NaN,NaN,N,N,3,Mountains,None,None
5,021,370210027021000,10,1,9,9,9,0,0,0,...,0.0,8.507039e+05,NaN,NaN,Y,Y,3,Mountains,None,None
6,021,370210027042036,14,5,9,7,6,0,0,0,...,0.0,6.177340e+03,NaN,NaN,Y,Y,3,Mountains,None,None
13,021,370210030024038,15,5,10,5,4,0,1,0,...,1.0,1.056996e+06,NaN,NaN,N,N,3,Mountains,None,None
15,021,370210026081008,33,3,30,30,25,4,0,1,...,0.0,NaN,NaN,NaN,N,N,3,Mountains,None,None
16,021,370210023051012,112,3,109,105,99,1,0,4,...,0.0,7.856433e+05,NaN,NaN,N,N,3,Mountains,None,None


In [1373]:
census_block_clusters_tiers_regions.to_file("census_block_clusters_tiers_regions_test.shp")

/var/folders/gr/tcb1m02d03zb8010bxpr4bfc0000gn/T/ipykernel_77152/358090181.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  census_block_clusters_tiers_regions.to_file("census_block_clusters_tiers_regions_test.shp")
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'block_total_area' to 'block_tota'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'area_of_cluster_muni_intersection' to 'area_of_cl'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'block_buffer_overlap_percent' to 'block_buff'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Pop_in_Prox_of_Polluter' to 'Total_Pop_'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogr

In [1374]:
# Creating the data 

In [1375]:
gdf = census_block_clusters_tiers_regions

In [1381]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
print(len(cell_gdf))


print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('All Clusters')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()))

27982
Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
All Clusters
Latino: 323695.57318109984
Asian: 65738.86885735286
Black: 829715.0514370039
Native American: 17084.75752700352
Total: 1469735.368511488


In [1382]:
cell_gdf = gdf[gdf['cluster_id'].notna()]

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('All Clusters, %')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_002N'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_008N'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_006N'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_007N'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_001N'].sum()))

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
All Clusters, %
Latino: 0.8430735835443429
Asian: 0.8175459377857588
Black: 0.8391699795971247
Native American: 0.3021444429569992
Total: 0.8174188098638991


In [1383]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']
print(len(cell_gdf))


print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('Unincorporated Clusters Near Any Municipality')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()))

4718
Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
Unincorporated Clusters Near Any Municipality
Latino: 98392.23389463159
Asian: 14829.457617684064
Black: 192188.0145285391
Native American: 3936.5612166712804
Total: 361676.43316052086


In [1384]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']


print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('Unincorporated Clusters Near Any Municipality, %')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_002N'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_008N'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_006N'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_007N'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_001N'].sum()))

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
Unincorporated Clusters Near Any Municipality, %
Latino: 0.9328223316201634
Asian: 0.8402435048832264
Black: 0.9173871191605484
Native American: 0.7027063935507463
Total: 0.911597815149391


In [1385]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_white_muni'] == 'Y']
print(len(cell_gdf))


print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('Unincorporated Clusters Near White Municipality')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()))

875
Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
Unincorporated Clusters Near White Municipality
Latino: 20790.993828544502
Asian: 3459.575881813942
Black: 36210.32766120434
Native American: 239.32156861823427
Total: 72429.75700352709


In [1386]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_white_muni'] == 'Y']


print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race')
print('Unincorporated Clusters Near White Municipality, %')
print('Latino: ' + str(cell_gdf['Total_Hispanic_or_Latino_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_002N'].sum()))
print('Asian: ' + str(cell_gdf['Total_NH_Asian_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_008N'].sum()))
print('Black: ' + str(cell_gdf['Total_NH_Black_AA_Pop_in_Prox_of_Polluter'].sum() / cell_gdf['P2_006N'].sum()))
print('Native American: ' + str(cell_gdf['Total_NH_Native_American_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_007N'].sum()))
print('Total: ' + str(cell_gdf['Total_Pop_in_Prox_of_Polluter'].sum()/ cell_gdf['P2_001N'].sum()))

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Race
Unincorporated Clusters Near White Municipality, %
Latino: 0.9039169526779054
Asian: 0.6310791466278625
Black: 0.8851433098145723
Native American: 0.8997051451813318
Total: 0.8700689162665725


In [1387]:
cell_gdf = gdf[gdf['cluster_id'].notna()]

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier')
print('All Clusters')
print(gdf.groupby(['Cluster_Tier'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['Cluster_Tier'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['Cluster_Tier'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier
All Clusters
Cluster_Tier
1    327519.655687
2    428228.802385
3    713986.910439
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
Cluster_Tier
1    0.643733
2    0.881262
3    0.888805
dtype: float64


In [1388]:
print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier')
print('General Population')
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['tier'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier
General Population
tier
1    1.142488e+06
2    2.294178e+06
3    3.390884e+06
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
tier
1    0.507183
2    0.612526
3    0.693130
dtype: float64


In [1389]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier')
print('Unincorporated Clusters Near Any Municipality')
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['tier'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier
Unincorporated Clusters Near Any Municipality
tier
1    1.142488e+06
2    2.294178e+06
3    3.390884e+06
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
tier
1    0.507183
2    0.612526
3    0.693130
dtype: float64


In [1390]:
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier')
print('Unincorporated Clusters Near Any Municipality')
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['tier'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['tier'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Tier
Unincorporated Clusters Near Any Municipality
tier
1    1.142488e+06
2    2.294178e+06
3    3.390884e+06
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
tier
1    0.507183
2    0.612526
3    0.693130
dtype: float64


In [1391]:
cell_gdf = gdf[gdf['cluster_id'].notna()]

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region')
print('All Clusters')
print(gdf.groupby(['Cluster_Region'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['Cluster_Region'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['Cluster_Region'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region
All Clusters
Cluster_Region
Coastal Plains    3.346516e+05
Mountains         1.240185e+04
The Piedmont      1.122682e+06
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
Cluster_Region
Coastal Plains    0.649825
Mountains         0.600603
The Piedmont      0.889335
dtype: float64


In [1392]:
print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region')
print('General Population')
print(gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum() / gdf.groupby(['regions'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region
General Population
regions
Coastal Plains    1.470364e+06
Mountains         6.082890e+05
The Piedmont      4.748897e+06
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
regions
Coastal Plains    0.509994
Mountains         0.507330
The Piedmont      0.697538
dtype: float64


In [1393]:
cell_gdf = gdf[gdf['cluster_id'].notna()]
cell_gdf = cell_gdf[cell_gdf['cluster_with_less_than_50pct_overlap_with_municipality'] == 'Y']
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region')
print('Unincorporated Clusters Near Any Municipality')
print(cell_gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(cell_gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum() / cell_gdf.groupby(['regions'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region
Unincorporated Clusters Near Any Municipality
regions
Coastal Plains     43275.70246
Mountains           1996.00000
The Piedmont      316404.73070
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
regions
Coastal Plains    0.817973
Mountains         1.000000
The Piedmont      0.925571
dtype: float64


In [1394]:
cell_gdf = cell_gdf[cell_gdf['near_muni'] == 'Y']

print('Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region')
print('All Census Blocks Near Municipalities')
print(cell_gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum())
print(cell_gdf.groupby(['regions'])['Total_Pop_in_Prox_of_Polluter'].sum() / cell_gdf.groupby(['regions'])['P2_001N'].sum())

Proximity of Cluster Residents to EPA‐Monitored Polluting Sites by Region
All Census Blocks Near Municipalities
regions
Coastal Plains     43275.70246
Mountains           1996.00000
The Piedmont      316404.73070
Name: Total_Pop_in_Prox_of_Polluter, dtype: float64
regions
Coastal Plains    0.817973
Mountains         1.000000
The Piedmont      0.925571
dtype: float64


In [1395]:
census_block_clusters_tiers_regions.to_file("census_block_clusters_tiers_regions_test2.shp")

/var/folders/gr/tcb1m02d03zb8010bxpr4bfc0000gn/T/ipykernel_77152/1879170916.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  census_block_clusters_tiers_regions.to_file("census_block_clusters_tiers_regions_test2.shp")
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'block_total_area' to 'block_tota'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'area_of_cluster_muni_intersection' to 'area_of_cl'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'block_buffer_overlap_percent' to 'block_buff'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyogrio/raw.py:709: RuntimeWarning: Normalized/laundered field name: 'Total_Pop_in_Prox_of_Polluter' to 'Total_Pop_'
  ogr_write(
/opt/homebrew/lib/python3.9/site-packages/pyo